In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd
import seaborn as sns; print(sns.__version__)

#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))


print(data_dir_input, scripts_dir)

subjects = [f for f in os.listdir(data_dir_input) if f.startswith('sub') ]
subjects = list(sorted(subjects))
print(subjects)

In [ ]:
%%javascript
// needed to access notebook path later
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

In [ ]:
print("NOTEBOOK_FULL_PATH:\n", NOTEBOOK_FULL_PATH)

In [ ]:
import os

def getAddInfo():
    import datetime
    # Get the current date and time in the format YYYY-MM-DD HH:MM:SS
    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # Get the name of the current jupyter notebook
    #notebook_name = os.path.basename(os.environ.get("NOTEBOOK_PATH", ""))
    notebook_name = NOTEBOOK_FULL_PATH
    #print(notebook_name)
    add_ttle = f"{now}\n{notebook_name}"
    return add_ttle

def addTitleInfo(axs):
    add_ttle = getAddInfo()
    # Add a linebreak and the date, time, and notebook name to the ylabel of each axis
    if axs is not None:
        if isinstance(axs, (list, np.ndarray)): # If axs is a list or an array of axes
            for ax in axs:
                title = ax.get_title()
                new_title = f"{title}\n{add_ttle}"
                ax.set_title(new_title)
        else: # If axs is a single axis
            title = axs.get_title()
            new_title = f"{title}\n{add_ttle}"
            axs.set_title(new_title)
    else:
        title = plt.gcf()._suptitle.get_text()
        new_title = f"{title}\n{add_ttle}"
        plt.suptitle(new_title)

In [ ]:
from config2 import path_data
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from behav_proc import *

import numpy as np
#from collections import OrderedDict as odict
from config2 import envcode2env
from pingouin import ttest

from base2 import radius_cursor
from datetime import datetime
radius_cursor

In [ ]:
fnf = pjoin(path_data,'df_all_multi_tsz.pkl.zip')
print(fnf)
print( str(datetime.fromtimestamp(os.stat(fnf).st_mtime)))
df_all_multi_tsz = pd.read_pickle(fnf)
#dfc_multi_tsz = pd.   read_pickle(pjoin(path_data,'df_all_multi_tsz_trunc_q=0.05.pkl.zip'))
#df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe"').copy()
#df_orig = df
#del df_all_multi_tsz

In [ ]:
#df_all_multi_tsz_whtr = df_all_multi_tsz.merge(stds, on='subject')
df = df_all_multi_tsz.\
    query('trial_group_col_calc in ["trialwe", "trialwtgt_we"]').copy()

df['env'] = df['environment'].apply(lambda x: envcode2env[x])
df['feedback_deg'] = df['feedback'] / np.pi * 180

def f(x):    
    if x > np.pi:
        x -= 2*np.pi
    elif x < -np.pi:
        x += 2*np.pi
    return x
df['error'] = df['error'].apply(f)

df['error_deg'] = df['error'] / np.pi * 180 

def f(x):    
    if x > np.pi:
        x -= 2*np.pi
    elif x < -np.pi:
        x += 2*np.pi
    return x
df['prev_error'] = df['prev_error'].apply(f)

df['env'] = df['env'].astype('category')
df['subject'] = df['subject'].astype('category')

print( df['pert_stage_wb'].unique() )

df['pert_stage_wb'] = df['pert_stage_wb'].where(
    df['env'] =="stable", -1 )
df['pert_stage_wb'] = df['pert_stage_wb'].astype(int)

df['pert_stage'] = df['pert_stage'].where(
    df['env'] =="stable", -1 )
df['pert_stage'] = df['pert_stage'].astype(int)

print( df['pert_stage'].unique() )

df['ps_'] = 'rnd'
c = df['pert_stage_wb'].isin([1,3])
df.loc[c,'ps_'] = 'pert' 
c = df['pert_stage_wb'].isin([2,4])
df.loc[c,'ps_'] = 'washout' 
c = df['pert_stage_wb'].isin([0])
df.loc[c,'ps_'] = 'pre' 
print( df['ps_'].unique() )

df['ps2_'] = 'rnd'
c = df['pert_stage_wb'].isin([1])
df.loc[c,'ps2_'] = 'pert_pro' 
c = df['pert_stage_wb'].isin([3])
df.loc[c,'ps2_'] = 'pert_contra' 
c = df['pert_stage_wb'].isin([2,4])
df.loc[c,'ps2_'] = 'washout' 
c = df['pert_stage_wb'].isin([0])
df.loc[c,'ps2_'] = 'pre' 
print( df['ps2_'].unique() )

df['trialwpertstage_wb'] = df['trialwpertstage_wb'].where(df['env'] =="stable", 
                                    df['trialwb'])
df['trialwpertstage_wb'] = df['trialwpertstage_wb'].astype(int)

# just get perturabtion and env scheme from one subject
# will be needed for plotting time resovled
#dfall = dfall.reset_index()
dfc_p = df.query(f'subject == "{subjects[0]}"')
dfc_p = dfc_p.sort_values('trials')
pert = dfc_p['perturbation'].values[:192*4]
tr   = dfc_p['trials'].values[:192*4]
envv = dfc_p['environment'].values[:192*4].astype(float)
envv[envv == 0] = np.nan
pert[envv == 1] = np.nan

In [ ]:
assert not df.duplicated(['subject','trials','trial_shift_size','trial_group_col_calc']).any()

In [ ]:
df[['subject','trials','trial_shift_size','trial_group_col_calc','err_sens']]

In [ ]:
stds

In [ ]:
# estimate error at second halfs of init stage
qs_initstage = 'pert_stage_wb.abs() < 1e-10'
df_init = df.query(qs_initstage + ' and trialwb >= 10')
grp = df_init.groupby(['subject','trial_shift_size','trial_group_col_calc'])
#display(grp.size())
#df_init['feedback_deg'].min(), df_init['feedback_deg'].max()

grp['error_deg'].std()

stds = grp['error_deg'].std()#.std()
mestd = stds.mean()
print(mestd, stds.std() )

stds = stds.to_frame().reset_index().rename(columns={'error_deg':'error_deg_initstd'})
df_wthr = df.merge(stds, on=['subject','trial_shift_size','trial_group_col_calc'])
#df_wthr

In [ ]:
df_wthr.trial_group_col_calc.unique(), df_wthr.trial_shift_size.unique()

In [ ]:
shiftszs = df['trial_shift_size'].unique()
shiftszs

In [ ]:
[col for col in df_all_multi_tsz.columns if col.find('like') >= 0]

In [ ]:
df_all_multi_tsz.query('trials <= 4 and subject == @subjects[0] and  trial_group_col_calc == "trialwe"')\
    [['trials','trial_shift_size','error','prev_error']]

In [ ]:
NN=5; st=0.4 #maxmult=1.2
#thrs =  np.linspace(mestd/NN, mestd, NN); print(mestd, thrs)
mults = st + (1-st) * np.arange(NN) / ( (NN - 1) )
mults = [0] + list(mults)
#thrs = mestd * mults
mults#, thrs

In [ ]:
# pooled random vs stable
dfs = []
ttrs = []
for shiftsz in shiftszs:
    for tgcc in df_wthr.trial_group_col_calc.unique():
        for multi, mult in enumerate(mults):
            # prev_error is not enough, it does not respect shift size
            df_ = df_wthr.query('trial_shift_size == @shiftsz and trial_group_col_calc == @tgcc').copy()
            assert not df_.duplicated(['subject','trials']).any()
            df_ = df_[df_.error_deg.shift(shiftsz).abs() >= \
                    df_['error_deg_initstd'] * mults[multi] ]        
            #df_ = df_.query('error_deg.shift(1).abs() >= @thr')
            #thr_s = '{:.3f}'.format(thr)
            #thr_s = f'{maxmult}*mestd/{ NN - (thri)}'
            #thr_s = f'mestd/{ NN - (thri)}'
            thr_s = f'mestd*{mults[multi]}'
            df_['thr'] = thr_s
            dfs += [df_]
            print(thr_s, len(df_))
dfall_notclean = pd.concat(dfs)
# ttrs = pd.concat(ttrs)
# ttrs = ttrs.rename(columns={'p-val':'pval'})

from behav_proc import truncateDf
dfall_mshsz  = truncateDf(dfall_notclean, 'err_sens', q=0.0, infnan_handling='discard', 
                   cols_uniqify = ['subject','environment','thr','trial_shift_size','trial_group_col_calc'])


### basic violin stats (random vs env and pert stages)

In [ ]:
sns.set(font_scale=1.3)

In [ ]:
print(getAddInfo())
thrs = ["mestd*1.0", "mestd*0"]
qs = 'thr in @thrs'
display( ttrs.query('pval < 0.05 and thr in @thrs') )

In [ ]:
# non pooled (av within participant) random vs stable
dfall_ = dfall_mshsz.query('trial_shift_size == 1')
me = dfall_.groupby(['trial_group_col_calc', 'thr','subject','env']).\
    mean(numeric_only=1).reset_index()
#me.groupby(['env','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me, kind='violin', y='err_sens', 
    x='env', col='thr', order=['stable','random'], row='trial_group_col_calc')
addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
    

In [ ]:
%matplotlib inline

In [ ]:
# -------------
me = dfall_.groupby(['trial_group_col_calc', 'thr','subject','block_name']).\
    mean(numeric_only=1).reset_index()
#display(me.groupby(['block_name','thr']).size())
sns.set(font_scale=1.3)
fg = sns.catplot(data = me, kind='violin', y='err_sens', 
    x='block_name', col='thr', order=['stable1','stable2',
                                     'random1','random2'], row='trial_group_col_calc')
addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
# -------------
me = dfall_.groupby(['trial_group_col_calc', 'thr','subject','pert_stage']).\
    mean(numeric_only=1).reset_index()
#me.groupby(['env','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me, kind='violin', y='err_sens', 
    x='pert_stage', col='thr', row='trial_group_col_calc')
addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)

In [ ]:
# non-pooled stages and env comparison
me = dfall_.groupby(['trial_group_col_calc', 'thr','subject','ps_']).\
    mean(numeric_only=1).reset_index()
#me.groupby(['ps_','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me, kind='violin', y='err_sens', 
    x='ps_', col='thr', row='trial_group_col_calc')
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
addTitleInfo(fg.axes.flatten()[0])

me = dfall_.groupby(['trial_group_col_calc','thr','subject','ps2_']).\
    mean(numeric_only=1).reset_index()

#me.groupby(['ps_','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me, kind='violin', y='err_sens', 
    x='ps2_', col='thr', row='trial_group_col_calc')
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
addTitleInfo(fg.axes.flatten()[0])

### Fig 2 CD

In [ ]:
dfall.subject.nunique()

In [ ]:
me.subject.nunique()

In [ ]:
thr = "mestd*1.0"
me = dfall.groupby(['thr','subject','env']).\
    mean(numeric_only=1).reset_index()
me_env = me
#me.groupby(['env','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me.query('thr == @thr'), 
                 kind='violin', y='err_sens', 
    x='env', col='thr', order=['stable','random'])
addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
    
ax.annotate('C', xy=(0, 1), xytext=(-60, 60), 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')
fg.set_ylabels('Error sensitivity')

plt.show()
    
###################

# non-pooled stages and env comparison
me = dfall.groupby(['thr','subject','ps_']).\
    mean(numeric_only=1).reset_index()
me_ps = me
#me.groupby(['ps_','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me.query('thr == @thr'), kind='violin', y='err_sens', 
    x='ps_', col='thr', order=['pre','pert','washout','rnd'])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
addTitleInfo(fg.axes.flatten()[0])
#plt.gcf().add_subplot_labels(['C'])

ax.annotate('D', xy=(0, 1), xytext=(-60, 60), 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')

fg.set_ylabels('Error sensitivity')
#fg.set_titles()

In [ ]:
statcols_toshow = ['varn','T','pval','val1',
                   'val2','alternative','pooled', 'ttstr','dof']

In [ ]:
for col, me in [('env', me_env), ('ps_', me_ps)]:
    for qs in ['thr == "mestd*1.0"', 'thr == "mestd*0"']:
        df_ = dfall #me.query(qs)
        print(col, qs)
        varn = 'err_sens'
        ttrssig, ttrs = comparePairs(df_,varn,col)
        display(ttrssig.query('alternative != "two-sided"')\
                [statcols_toshow])

### Fig 2 AB

In [ ]:
c = 1 / np.pi  * 180
thrs = ["mestd*1.0"]#, "mestd*0"]
qs = 'thr in @thrs'
for lablet,varn,pertc in [('A', 'error_pscadj', 1.), 
                   ('B', 'err_sens', 0.06)]:
    #varn = 'org_feedback'
    #for varn in     
    #for truncs, df in zip(['truncate q=5%','truncate no'],[dfc, dfc0]):
    #for truncs, dfall in zip(['truncate q=5%'],[df_]):
    varn_eff = varn
    if varn.startswith('org_feedback'):
        dfall['vtp'] = np.array(dfall[varn]) * c# - 180
        varn_eff = 'vtp'
    elif varn.startswith('error'):
        dfall['vtp'] = np.array(dfall[varn]) * c 
        varn_eff = 'vtp'
    #elif varn.startswith('err_sens'):
    #    pert_shift = 
        #df['vtp'] = np.array(df[varn]) 
        #varn_eff = 'vtp'

    plt.figure(figsize=(12,4))
    fg = sns.relplot(data=dfall.query(qs),
        kind='line', x="trials", y=varn_eff, 
        col='thr', estimator='mean', ci='sd', height = 4, aspect=2.5)

    for ax in fg.axes.flatten():
        ax.plot(tr, pert * pertc, c='violet', ls= '--')
        ax.plot(tr, envv * 30 * pertc, c='black', lw=0, marker='.' ,alpha=0.2)
        #ax.set_title(f'{varn}  {truncs}')
        ax.set_ylabel(varn )
    if varn.startswith('error_'):
        fg.set_ylabels('Angular error [deg]')
        fg.set_titles('Error dynamics')
    else:
        fg.set_ylabels('Error sensitivity')
        fg.set_titles('Error sensitivity dynamics')
    fg.set_xlabels('Trial number')
    
     
    #addTitleInfo(fg.axes.flatten()[0])
    ax.annotate(lablet, xy=(0, 1), xytext=(-60, 20), 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')

In [ ]:
# ttests stages and envs

In [ ]:
# compare all pairs stats
from behav_proc import comparePairs    
df_ = dfall.query('thr == "mestd*0"')
varn = 'err_sens'
col = 'ps_'

ttrssig, ttrs = comparePairs(df_,varn,col)
ttrssig[statcols_toshow]

# per > pert
# pre > washout
# pre > rnd
# pert < washout
# washout > rnd

# pert < rnd only in pooled (pval is 0.028, much bigger than others)

In [ ]:
#df_.groupby(['subject','ps_']).mean(numeric_only = 1)

## compare first and second random ES -- no difference

In [ ]:

from behav_proc import comparePairs    
df_ = dfall.query('thr == "mestd*0" and env == "random"')
varn = 'err_sens'
col = 'block_name'

print('Compare different random blocks no trunc')
ttrssig, ttrs = comparePairs(df_,varn,col)
if ttrssig is None:
    print('None')
else:
    ttrssig[statcols_toshow]

print('Compare different random blocks')
df_ = dfall.query('thr == "mestd*1.0" and env == "random"')
varn = 'err_sens'
col = 'block_name'

ttrssig, ttrs = comparePairs(df_,varn,col)
if ttrssig is None:
    print('None')
else:
    ttrssig[statcols_toshow]

In [ ]:
import gc; gc.collect()

In [ ]:
import pingouin as pg

# Time resolved

In [ ]:
c = 1 / np.pi  * 180
thrs = ["mestd*1.0", "mestd*0"]
qs = 'thr in @thrs'
for varn,pertc in [('error_pscadj', 1.), ('err_sens', 0.04)]:
    #varn = 'org_feedback'
    #for varn in     
    #for truncs, df in zip(['truncate q=5%','truncate no'],[dfc, dfc0]):
    #for truncs, dfall in zip(['truncate q=5%'],[df_]):
    varn_eff = varn
    if varn.startswith('org_feedback'):
        dfall['vtp'] = np.array(dfall[varn]) * c# - 180
        varn_eff = 'vtp'
    elif varn.startswith('error'):
        dfall['vtp'] = np.array(dfall[varn]) * c 
        varn_eff = 'vtp'
    #elif varn.startswith('err_sens'):
    #    pert_shift = 
        #df['vtp'] = np.array(df[varn]) 
        #varn_eff = 'vtp'

    plt.figure(figsize=(12,4))
    fg = sns.relplot(data=dfall.query(qs), kind='line', x="trials", y=varn_eff, col='thr', estimator='mean', ci='sd')

    for ax in fg.axes.flatten():
        ax.plot(tr, pert * pertc, c='violet', ls= '--')
        ax.plot(tr, envv * 30 * pertc, c='black', lw=0, marker='.' ,alpha=0.2)
        #ax.set_title(f'{varn}  {truncs}')
        ax.set_ylabel(varn )
    # sns.lineplot(ax=ax, x="trials", y="perturbation",
    #              data=dfc.query('pert_seq_code == 0') )

    #plt.legend(title=None, loc='lower right',
    #           labels=['within tgt', '_nolegend_', 'across tgt'])
    #ax.set_ylim(-0.25,0.3)
for ax in fg.axes.flatten():
    ax.set_ylim(-5,5)

# classical corr (separman) trialwpertstage_wb

### random

In [ ]:
dfallrnd = dfall.query('env == "random"').copy()
dfallrnd['error_abs'] = dfallrnd['error'].abs()
print('random: err_abs')
def f(df_):
    r = pg.corr( df_['trialwb'], 
        df_['error_abs'],  method='spearman')
    return r
corrs = dfallrnd.\
    groupby(['thr','block_name']).apply(f)
corrs = corrs.rename(columns={'p-val':'pval'})
#display(corrs)

print(getAddInfo())
thrs = ["mestd*1.0", "mestd*0"]
qs = 'thr in @thrs'
display( corrs.query('pval < 0.05 and thr in @thrs') )

print('random: err_sens')
def f(df_):
    r = pg.corr( df_['trialwb'], 
        df_['err_sens'],  method='spearman')
    return r
corrs = dfallrnd.\
    groupby(['thr','block_name']).apply(f)
corrs = corrs.rename(columns={'p-val':'pval'})
#display(corrs)

print(getAddInfo())
thrs = ["mestd*1.0", "mestd*0"]
qs = 'thr in @thrs'
display( corrs.query('pval < 0.05 and thr in @thrs') )

In [ ]:
corrs_per_subj = dfallrnd.query('thr == "mestd*0"').\
    groupby(['thr','subject','block_name']).apply(f)

In [ ]:
corrs_per_subj = corrs_per_subj.rename(columns={'p-val':'pval'})
print(getAddInfo())
with pd.option_context('display.min_rows',40):
    display(\
        corrs_per_subj.query(qs + ' and pval < 0.05').groupby(['subject']).size()
        )

In [ ]:
gc.collect()

## abs error and trial

In [ ]:
#df_['error_pscadj_pertstageadj']

In [ ]:
# stable
dfall['error_pscadj_pertstageadj_abs'] = dfall['error_pscadj_pertstageadj'].abs()

#dfallst = dfall.query('env == "stable"').copy()
dfallst = dfall.copy()
dfallst['pert_stage'] = dfallst['pert_stage'].astype('int').astype('category')
dfallst['pert_stage_wb'] = dfallst['pert_stage_wb'].astype('int').astype('category')

# error
def f(df_):
    r = pg.corr( df_['trialwpertstage_wb'], 
                df_['error_pscadj_pertstageadj_abs'],  method='spearman')
    return r
corrs_per_subj = dfallst.query('thr == "mestd*0"').\
    groupby(['thr','subject','pert_stage_wb']).apply(f)
corrs_per_subj = corrs_per_subj.rename(columns={'p-val':'pval'})
corrs_per_subj_me = corrs_per_subj.\
    groupby('pert_stage_wb')[['r','pval']].mean()
display(corrs_per_subj_me)

corrs_per_subj_me_errabs = corrs_per_subj_me

#corrs = dfallst.query('thr == "mestd*0"').\
#    groupby(['thr','pert_stage']).apply(f)
corrs = dfallst.\
    groupby(['thr','pert_stage_wb']).apply(f)

corrs = corrs.rename(columns={'p-val':'pval'})
display(corrs)
# def f(df_):    
#     a = df_.query('env == "stable"')['r']
#     b = df_.query('env == "random"')['r']
#     rs = []
#     for alt in ['two-sided','greater','less']:
#         print(len(a),len(b), a ,b)
#         rs.append( pg.ttest( a, b, alternative = alt) )
#     return pd.concat(rs)
# ttest_of_corr = corrs_per_subj.groupby(['thr']).apply(f)
# display(ttest_of_corr)

In [ ]:
print(getAddInfo())
thrs = ["mestd*1.0", "mestd*0"]
qs = 'thr in @thrs'
display( corrs.query('pval < 0.05 and thr in @thrs') )

In [ ]:
# all corrs are negative within participants
def f(df):
    if len(df) == 0:
        return None
    ttrs = compare0(df, 'r')
    return ttrs
ttrs = corrs_per_subj.query('pert_stage_wb != -1').\
    groupby(['thr','pert_stage_wb']).apply(f)
ttrs.query('pval <= 0.05')

In [ ]:
print(getAddInfo())
with pd.option_context('display.min_rows',40):
    display(\
        corrs_per_subj.query(qs + ' and pval < 0.05').groupby(['pert_stage_wb']).size()
        )

## err sens and trial

In [ ]:
def f(df_):
    r = pg.corr( df_['trialwpertstage_wb'], df_['err_sens'],  method='spearman')
    r['method'] = 'spearman'
    return r
corrs_per_subj = dfallst.groupby(['thr','subject','pert_stage_wb']).apply(f)
corrs_per_subj = corrs_per_subj.rename(columns={'p-val':'pval'})
display(corrs_per_subj)

corrs_per_subj_me = corrs_per_subj.\
    groupby(['thr','pert_stage_wb'])[['r','pval']].mean()
corrs_per_subj_me['method'] = 'spearman'
corrs_per_subj_me_ES = corrs_per_subj_me

corrs_per_subj2 = dfallst.groupby(['thr','subject','ps_']).apply(f)
corrs_per_subj2 = corrs_per_subj2.rename(columns={'p-val':'pval'})
corrs_per_subj2_me = corrs_per_subj2.\
    groupby(['thr','ps_'])[['r','pval']].mean()
corrs_per_subj2_me['method'] = 'spearman'
corrs_per_subj2_me_ES = corrs_per_subj2_me

corrs = dfallst.groupby(['thr','pert_stage_wb']).apply(f)
corrs = corrs.rename(columns={'p-val':'pval'})
display(corrs)



def f(df_):
    try:
        r = pg.corr( df_['trialwpertstage_wb'], df_['err_sens'],  
                    method='pearson')
        r['method'] = 'pearson'
        r['mean_x'] = df_['trialwpertstage_wb'].mean()
        r['mean_y'] = df_['err_sens'].mean()
        r['std_x'] = df_['trialwpertstage_wb'].std()
        r['std_y'] = df_['err_sens'].std()
    except ValueError as e:
        return None
    return r
corrs_per_subj_me_pearson = dfallst.groupby(['thr','subject','pert_stage_wb']).apply(f)
corrs_per_subj_me_pearson = corrs_per_subj_me_pearson.rename(columns={'p-val':'pval'})
corrs_per_subj_me_pearson = corrs_per_subj_me_pearson.\
    groupby(['thr','pert_stage_wb'])[['r','pval',
        'mean_x','mean_y','std_x','std_y']].mean()
corrs_per_subj_me_pearson['method'] = 'pearson'
corrs_per_subj_me_ES_pearson = corrs_per_subj_me_pearson

In [ ]:
corrs_per_subj2_me

In [ ]:
#corrs_per_subj_me_ES_pearson

In [ ]:
print(getAddInfo())
with pd.option_context('display.min_rows',40):
    display(\
        corrs_per_subj.query(qs + ' and pval < 0.05').groupby(['pert_stage_wb']).size()
        )

In [ ]:
# pooled
print(getAddInfo())
thrs = ["mestd*1.0", "mestd*0"]
qs = 'thr in @thrs'
display( corrs.query('pval < 0.05 and thr in @thrs') )

In [ ]:
# all corrs are positive within participants
from behav_proc import compare0
def f(df):
    if len(df) == 0:
        return None
    ttrs = compare0(df, 'r')
    return ttrs
ttrs = corrs_per_subj.\
    groupby(['thr','pert_stage_wb']).apply(f)
print(getAddInfo())
display(ttrs.query('pval <= 0.05 and thr in @thrs'))

ttrs = corrs_per_subj2.\
    groupby(['thr','ps_']).apply(f)
print(getAddInfo())
display(ttrs.query('pval <= 0.05 and thr in @thrs'))

In [ ]:
#ttest_of_corr[ttest_of_corr['p-val'] < 5e-2]

# partial corr

In [ ]:
dfs = []
dfall['error_abs'] = dfall['error'].abs()
dfall['prev_error_abs'] = dfall['prev_error'].abs()

dfrs_per_subj = []
dfrs_per_subj2 = []
for varn in ['error','error_abs','prev_error','prev_error_abs']:
    method = 'spearman'
    def f(df_):
        if len(df_) < 3:
            return pd.DataFrame()
        r = pg.partial_corr( df_, 'trialwpertstage_wb', 'err_sens', varn,  
                            method=method)
        return r
    
    dfall_  = dfall.query('pert_stage_wb != -1')

    pcorrs = dfall_.\
        groupby(['thr','pert_stage_wb']).apply(f)
    pcorrs['varn'] = varn
    pcorrs['method'] = method
    pcorrs = pcorrs.rename(columns={'p-val':'pval'})
    dfs += [pcorrs]
    #sigcorrs = pcorrs[pcorrs['pval'] < 5e-2]
    #dfs += [sigcorrs]
    
    method = 'pearson'
    def f(df_):
        if len(df_) < 3:
            return pd.DataFrame()
        r = pg.partial_corr( df_, 'trialwpertstage_wb', 
                            'err_sens', varn,  
                            method=method)
        r['mean_x'] = df_['trialwpertstage_wb'].mean()
        r['mean_y'] = df_['err_sens'].mean()
        r['std_x'] = df_['trialwpertstage_wb'].std()
        r['std_y'] = df_['err_sens'].std()
        r['mean_z'] = df_[varn].mean()
        return r
    pcorrs_per_subj = dfall_.groupby(['thr','subject','pert_stage_wb']).apply(f)
    #display(pcorrs_per_subj)
    pcorrs_per_subj['varn'] = varn
    pcorrs_per_subj['method'] = method
    pcorrs_per_subj = pcorrs_per_subj.rename(columns={'p-val':'pval'})
    dfrs_per_subj += [pcorrs_per_subj.reset_index()]
    
    pcorrs_per_subj2 = dfall_.groupby(['thr','subject','ps_']).apply(f)
    pcorrs_per_subj2 = pcorrs_per_subj2.rename(columns={'p-val':'pval'})
    pcorrs_per_subj2['varn'] = varn
    pcorrs_per_subj2['method'] = method
    
    dfrs_per_subj2 += [pcorrs_per_subj2.reset_index()]
#     pcorrs_per_subj2_me = pcorrs_per_subj2.\
#         groupby(['thr','ps_'])[['r','pval']].mean()
#     pcorrs_per_subj2_me_ES = pcorrs_per_subj2_me
    
    
    
dfr = pd.concat(dfs, ignore_index=1)#.set_index('')
dfr_per_subj = pd.concat(dfrs_per_subj, ignore_index=1)
dfr_per_subj2 = pd.concat(dfrs_per_subj2, ignore_index=1)

In [ ]:
pcorrs_per_subj = dfr_per_subj.query('varn == "error_abs"')
pcorrs_per_subj_me = pcorrs_per_subj.\
    groupby(['thr','pert_stage_wb'])[['r','pval',
        'mean_x','mean_y','std_x','std_y','mean_z']].mean()

pcorrs_per_subj2 = dfr_per_subj2.query('varn == "error_abs"')
pcorrs_per_subj2_me = pcorrs_per_subj2.\
    groupby(['thr','ps_'])[['r','pval',
        'mean_x','mean_y','std_x','std_y','mean_z']].mean()

In [ ]:
pcorrs_per_subj2_me

In [ ]:
pg.__version__

In [ ]:
gc.collect()

In [ ]:
# all varn
print(getAddInfo())
with pd.option_context('display.max_rows', 500):
    display(dfr.query('thr == "mestd*1.0" and pert_stage_wb in [1,3]'))
    display(dfr.query('thr == "mestd*1.0" and pert_stage_wb in [0,2,4]'))

In [ ]:
# only err abs
print(getAddInfo())
thrs = ["mestd*1.0", "mestd*0"]
qs = 'thr in @thrs'
with pd.option_context('display.max_rows', 500):
    dfr_ = dfr.query('varn == "error_abs"')
    display(dfr_.query(qs + ' and pert_stage_wb in [1,3]'))
    display(dfr_.query(qs + ' and pert_stage_wb in [0,2,4]'))

In [ ]:
dfr_per_subj

In [ ]:
# within participant 1 and 3 are neg, 4 is pos
from behav_proc import compare0
def f(df):
    if len(df) == 0:
        return None
    ttrs = compare0(df, 'r')
    return ttrs
ttrs = dfr_per_subj.query('varn == "error_abs"').\
    groupby(['thr','pert_stage_wb']).apply(f)
print(getAddInfo())
display(ttrs.query('pval <= 0.05 and thr in @thrs'))

ttrs = dfr_per_subj2.query('varn == "error_abs"').\
    groupby(['thr','ps_']).apply(f)
print(getAddInfo())
display(ttrs.query('pval <= 0.05 and thr in @thrs'))

### within pert stage

In [ ]:
dfall['error_abs'] = dfall['error'].abs()
def f(df_):
    if len(df_) < 3:
        return pd.DataFrame()
    r = pg.partial_corr( df_, 'trialwpertstage_wb', 'err_sens', 'error_abs',  method='spearman')    
    return r

In [ ]:
persubj = dfall.groupby(['thr','subject','pert_stage_wb']).apply(f)
   #['p-val']
persubj['signif'] = persubj['p-val'] < 0.05
persubj.query('signif').groupby(['thr','pert_stage_wb']).size()

In [ ]:
dfall['error_abs'] = dfall['error'].abs()
def f(df_):
    if len(df_) < 3:
        return pd.DataFrame()
    #r = pg.partial_corr( df_, 'trialwtgt_wpertstage_wb', 'err_sens', 'error_abs',  method='spearman')    
    r = pg.partial_corr( df_, 'trialwpertstage_wb', 'err_sens', 'error_abs',  method='spearman')    
    return r

In [ ]:
#pcorrs_per_subj = dfall.groupby(['thr','subject','env']).apply(f)
#display(pcorrs_per_subj)

print('pooled subj signif corr')
pcorrs_ = dfall.groupby(['thr','pert_stage_wb']).apply(f)
display(pcorrs_[pcorrs_['p-val'] < 0.05])

print('average of within subj')
pcorrs_per_subj_ = dfall.groupby(['thr','subject','pert_stage_wb']).apply(f).\
    groupby(['thr','pert_stage_wb'])['p-val'].mean().to_frame()
display(pcorrs_per_subj_[pcorrs_per_subj_['p-val'] < 0.05])

In [ ]:
pcorrs_per_subj_r = dfall.groupby(['thr','subject','pert_stage_wb']).apply(f).\
    groupby(['thr','pert_stage_wb'])['r'].mean().to_frame()
pcorrs_per_subj_r

In [ ]:
persubj = dfall.groupby(['thr','subject','pert_stage_wb']).apply(f)
   #['p-val']
persubj['signif'] = persubj['p-val'] < 0.05
persubj.query('signif').groupby(['thr','pert_stage_wb']).size()

In [ ]:
persubj = dfall.groupby(['thr','subject','pert_stage']).apply(f)
   #['p-val']
persubj['signif'] = persubj['p-val'] < 0.05
persubj.query('signif').groupby(['thr','pert_stage']).size()

In [ ]:
persubj = dfall.groupby(['thr','subject','environment']).apply(f)
   #['p-val']
persubj['signif'] = persubj['p-val'] < 0.05
persubj.query('signif').groupby(['thr','environment']).size()

In [ ]:
pcorrs_per_subj_ = dfall.groupby(['thr','subject','pert_stage']).apply(f).\
    groupby(['thr','pert_stage'])['p-val'].mean().to_frame()
display(pcorrs_per_subj_[pcorrs_per_subj_['p-val'] < 0.05])

In [ ]:
# def f(df_):
#     a = df_.query('env == "stable"')['r']
#     b = df_.query('env == "random"')['r']
#     rs = []
#     for alt in ['two-sided','greater','less']:
#         rs.append( pg.ttest( a, b, alternative = alt) )
#     return pd.concat(rs)
# ttest_of_pcorr = pcorrs_per_subj.groupby(['thr']).apply(f)
# display(ttest_of_pcorr)

pcorrs_ = dfall.groupby(['thr','pert_stage']).apply(f)#.to_frame()
display(pcorrs_[pcorrs_['p-val'] < 0.05])

# corr with abs error for plot

In [ ]:
# correlation between ES and abs err within env
dfall['error_abs'] = dfall['error'].abs()
dfall['prev_error_abs'] = dfall['prev_error'].abs()
dfs = []
for varn in ['error','error_abs','prev_error','prev_error_abs']:
    def f(df_):
        #df_.query('err_sens < inf')
        r = pg.corr( df_[varn], df_['err_sens'],  method='spearman')
        return r
    corrs_abserr_per_subj = dfall.groupby(['thr','subject','env']).apply(f)
    #display(corrs_abserr_per_subj)

    corrs_abserr = dfall.groupby(['thr','env']).apply(f)
    #display(corrs_abserr)

    def f(df_):    
        a = df_.query('env == "stable"')['r']
        b = df_.query('env == "random"')['r']
        #print(len(a),len(b),a.abs().min(),b.abs().min())
        rs = []
        for alt in ['two-sided','greater','less']:
            #print(a.values, b.values)
            rs.append( pg.ttest( a, b, alternative = alt) )
        return pd.concat(rs)

    #%debug
    import warnings
    #warnings.filterwarnings("error")
    warnings.filterwarnings("ignore")
    #with warnings.catch_warnings(record=True) as w:
    ttest_of_corr_abserr = corrs_abserr_per_subj.\
        groupby(['thr']).apply(f)
    ttest_of_corr_abserr['varn'] = varn
    dfs += [ttest_of_corr_abserr]

dfr = pd.concat(dfs)
dfr = dfr.rename(columns={'p-val':'pval'})

corrs_abserr = corrs_abserr.rename(columns={'p-val':'pval'})

In [ ]:
corrs_abserr_per_subj

In [ ]:
print(getAddInfo())
#display( corrs[corrs['pval'] < 5e-2] )
with pd.option_context('display.max_rows', 500):
    dfr = dfr[dfr['pval'] <= 0.05]
    display(dfr.query('thr == "mestd*1.0"' ))# and alternative=="two-sided"'))
    #display(dfr.query('thr == "mestd*1.0" and pert_stage_wb in [0,2,4]'))

In [ ]:
print(getAddInfo())
thrs = ["mestd*1.0", "mestd*0"]
qs = 'thr in @thrs'
corrs_ = corrs_abserr.query(qs)
display( corrs_[corrs_['pval'] < 5e-2] )
with pd.option_context('display.max_rows', 500):
    dfr_ = dfr.query('varn == "error_abs" and pval < 0.05')
    display(dfr_.query(qs))
    #display(dfr_.query('thr == "mestd*1.0"))

In [ ]:
pg.__version__

In [ ]:
# signed error
def f(df_):
    r = pg.corr( df_['error'], df_['err_sens'],  method='spearman')
    return r
corrs_err_per_subj = dfall.groupby(['thr','subject','env']).apply(f)


# preverror
def f(df_):
    r = pg.corr( df_['prev_error_abs'], df_['err_sens'],  method='spearman')
    return r
corrs_prevabserr_per_subj = dfall.groupby(['thr','subject','env']).apply(f)

def f(df_):    
    a = df_.query('env == "stable"')['r']
    b = df_.query('env == "random"')['r']
    rs = []
    for alt in ['two-sided','greater','less']:
        rs.append( pg.ttest( a, b, alternative = alt) )
    return pd.concat(rs)
ttest_of_corr_prevabserr = corrs_prevabserr_per_subj.groupby(['thr']).apply(f)
display(ttest_of_corr_prevabserr)

In [ ]:
#dfall.set_index()

In [ ]:
len(dfall), len(dfall_aug)

In [ ]:
#np.cov( df_['error_abs'].values[:10], df_['error_abs'].values[:10])

In [ ]:
def f(df_):
    #df_.query('err_sens < inf')
    r = np.cov( df_['err_sens'].values, df_['error'].values )
    return r[0,1] / r[1,1]
covs_err_per_subj = dfall.groupby(['thr','subject','env']).apply(f).to_frame().reset_index().rename(columns={0:'cov'})

def f(df_):
    #df_.query('err_sens < inf')
    r = np.cov( df_['err_sens'].values, df_['error_abs'].values )
    return r[0,1] / r[1,1]
covs_abserr_per_subj = dfall.groupby(['thr','subject','env']).apply(f).to_frame().reset_index().rename(columns={0:'covabs'})

def f(df_):
    #df_.query('err_sens < inf')
    r = np.cov( df_['err_sens'].values, df_['prev_error_abs'].values  )
    return r[0,1] / r[1,1]
covs_prevabserr_per_subj = dfall.groupby(['thr','subject','env']).apply(f)
covs_prevabserr_per_subj = covs_prevabserr_per_subj.to_frame().reset_index().rename(columns={0:'covabsprev'})

display(covs_err_per_subj)
display(covs_abserr_per_subj)
display(covs_prevabserr_per_subj)

In [ ]:
#dfall_aug.columns

In [ ]:
#dfall_aug.groupby(['thr','subject','env'])['r'].std().max()

In [ ]:
dfall_aug = dfall.merge(covs_abserr_per_subj, on = ['thr','subject','env'])
dfall_aug = dfall_aug.merge(covs_prevabserr_per_subj, on = ['thr','subject','env'])
dfall_aug = dfall_aug.merge(covs_err_per_subj, on = ['thr','subject','env'])

dfall_aug = dfall_aug.assign(err_sens_abserrcorr = \
            dfall_aug['err_sens'] - dfall_aug['error_abs'] * dfall_aug['covabs'])
dfall_aug = dfall_aug.assign(err_sens_errcorr = \
            dfall_aug['err_sens'] - dfall_aug['error'] * dfall_aug['cov'])
dfall_aug = dfall_aug.assign(err_sens_prevabserrcorr = \
            dfall_aug['err_sens'] - dfall_aug['prev_error_abs'] * dfall_aug['covabsprev'])

#dfall_aug = dfall_aug.drop(columns=['r', 'ES_y'])

In [ ]:
dfall_aug[['err_sens','err_sens_abserrcorr','cov', 'error_abs']]

In [ ]:
# zoom in
df_ = dfall_aug.query('thr == "mestd*1.0"').copy()
c = 1 / np.pi  * 180
for varn,pertc in [('error_pscadj', 1.), ('err_sens', 0.04), 
        ('err_sens_abserrcorr', 0.04), ('err_sens_errcorr', 0.04)]:
    #varn = 'org_feedback'
    #for varn in     
    #for truncs, df in zip(['truncate q=5%','truncate no'],[dfc, dfc0]):
    #for truncs, dfall in zip(['truncate q=5%'],[df_]):
    varn_eff = varn
    if varn.startswith('org_feedback'):
        df_['vtp'] = np.array(df_[varn]) * c# - 180
        varn_eff = 'vtp'
    elif varn.startswith('error'):
        df_['vtp'] = np.array(df_[varn]) * c 
        varn_eff = 'vtp'
    #elif varn.startswith('err_sens'):
    #    pert_shift = 
        #df['vtp'] = np.array(df[varn]) 
        #varn_eff = 'vtp'

    #plt.figure(figsize=(18,4))
    fg = sns.relplot(data=df_, kind='line', x="trials", y=varn_eff, col='thr',
                     estimator='mean', ci='sd', height=3, aspect=3)

    for ax in fg.axes.flatten():
        ax.plot(tr, pert * pertc, c='violet', ls= '--')
        ax.plot(tr, envv * 30 * pertc, c='black', lw=0, marker='.' ,alpha=0.2)
        #ax.set_title(f'{varn}  {truncs}')
        ax.set_ylabel(varn )
        ax.axhline(0,ls=':',c='red', alpha=0.7)
        addTitleInfo(ax)
    # sns.lineplot(ax=ax, x="trials", y="perturbation",
    #              data=dfc.query('pert_seq_code == 0') )

    #plt.legend(title=None, loc='lower right',
    #           labels=['within tgt', '_nolegend_', 'across tgt'])
    #ax.set_ylim(-0.25,0.3)
    if varn.startswith('err_sens'):
        for ax in fg.axes.flatten():
            ax.set_ylim(-3,3)
    else:
        for ax in fg.axes.flatten():
            ax.set_ylim(-35,35)

In [ ]:
df_ = dfall_aug
c = 1 / np.pi  * 180
for varn,pertc in [('error_pscadj', 1.), ('err_sens', 0.04),                    
        ('err_sens_abserrcorr', 0.04), 
                   ('err_sens_prevabserrcorr', 0.04),
                  ('err_sens_errcorr', 0.04),]:
    #varn = 'org_feedback'
    #for varn in     
    #for truncs, df in zip(['truncate q=5%','truncate no'],[dfc, dfc0]):
    #for truncs, dfall in zip(['truncate q=5%'],[df_]):
    varn_eff = varn
    if varn.startswith('org_feedback'):
        df_['vtp'] = np.array(df_[varn]) * c# - 180
        varn_eff = 'vtp'
    elif varn.startswith('error'):
        df_['vtp'] = np.array(df_[varn]) * c 
        varn_eff = 'vtp'
    #elif varn.startswith('err_sens'):
    #    pert_shift = 
        #df['vtp'] = np.array(df[varn]) 
        #varn_eff = 'vtp'

    #plt.figure(figsize=(12,4))
    fg = sns.relplot(data=df_, kind='line', x="trials", y=varn_eff, 
                     col='thr', estimator='mean', ci='sd', height=4)

    for ax in fg.axes.flatten():
        ax.plot(tr, pert * pertc, c='violet', ls= '--')
        ax.plot(tr, envv * 30 * pertc, c='black', lw=0, marker='.' ,alpha=0.2)
        #ax.set_title(f'{varn}  {truncs}')
        ax.set_ylabel(varn, fontsize = 20 )
        ax.axhline(0,ls=':',c='red', alpha=0.7)
    # sns.lineplot(ax=ax, x="trials", y="perturbation",
    #              data=dfc.query('pert_seq_code == 0') )

    #plt.legend(title=None, loc='lower right',
    #           labels=['within tgt', '_nolegend_', 'across tgt'])
    #ax.set_ylim(-0.25,0.3)
    if varn.startswith('err_sens'):
        for ax in fg.axes.flatten():
            ax.set_ylim(-5,5)
    
    addTitleInfo(fg.axes.flatten()[0])

In [ ]:
gc.collect()

# Within pert stage time resolved plots (Fig 3)

In [ ]:
dfall = dfall.reset_index()

In [ ]:
dfc = dfall_aug.query('thr == "mestd*1.0"')
dfc['trialwpertstage_wb'] = dfc['trialwpertstage_wb'].\
    where(dfc['environment'] == 0, dfc['trialwb'])
dfc['trialwpertstage_wb'] = dfc['trialwpertstage_wb'].astype(int)

dfc.duplicated(['subject','trials']).any()
# nan-ify after pause
dfc.loc[dfc['trialwb'] == 0, 'err_sens'] = np.nan

In [ ]:
sns.__version__

In [ ]:
dfall_notclean.query('thr == "mestd*1.0" and trialwpertstage_wb.isin([0,1]) and pert_stage_wb.isin([1,3])').\
    sort_values(['thr','subject',
            'pert_stage','trials'])\
    [['subject','trials','trialwb','pert_stage','err_sens','error']]

In [ ]:
# dfc.query('trialwb == 0')\
#     [['subject','trials','trialwb','block_name','err_sens','error']]

## Fig 3

In [ ]:
pswb2r = corrs_per_subj_me_ES_pearson.loc[('mestd*1.0',slice(None))].\
    to_dict()
print(pswb2r)
def f(row):
    ps = row['pert_stage_wb']
    r = pswb2r['r'][ps]
    std_x = pswb2r['std_x'][ps]
    mean_x = pswb2r['mean_x'][ps]
    std_y = pswb2r['std_y'][ps]
    mean_y = pswb2r['mean_y'][ps]
    xs = row['trialwpertstage_wb']
    return mean_y + r * (xs - mean_x) / std_x * std_y 
dfc['pred'] = dfc.apply(f, axis=1)
#corrs_per_subj_me_ES['r'] * dfc['error_abs']

pswb2r = pcorrs_per_subj_me.loc[('mestd*1.0',slice(None))].\
    to_dict()
print(pswb2r)
def f(row):
    ps = row['pert_stage_wb']
    if ps == -1:
        return None
    r = pswb2r['r'][ps]
    std_x = pswb2r['std_x'][ps]
    mean_x = pswb2r['mean_x'][ps]
    std_y = pswb2r['std_y'][ps]
    mean_y = pswb2r['mean_y'][ps]
    #mean_z =  pswb2r['mean_z'][ps]
    xs = row['trialwpertstage_wb']
    return mean_y + r * (xs - mean_x) / std_x * std_y# + mean_z
dfc['ppred'] = dfc.apply(f, axis=1)

In [ ]:
# dfc['pred'] = corrs_per_subj_me_ES['r'] * dfc['error_abs']
fdsfsd

In [ ]:
#fg.axes_dict.keys()
#dict_keys(['pre', 'pert', 'washout', 'rnd'])
#fg.

In [ ]:
fg.col_names

In [ ]:
hue_order = dfc['pert_stage_wb'].unique()
col_order = ['pre', 'pert', 'washout', 'rnd']
hues = [[0],[1,3],[2,4],[5]]

# TODO: add fit
# TODO: start of pert
# TODO: sd instead of se
palette=['blue', 'orange', 'green', 'olive','cyan','brown']
df_ = dfc.query('trial_shift_size == 1')
fg = sns.relplot(data=df_, kind='line',
            x='trialwpertstage_wb', col='ps_',
           y='err_sens', hue='pert_stage_wb',
                 ci = 'sd', palette = palette,
           facet_kws={'sharex':False},
                 hue_order=hue_order,
            col_order = col_order, legend=None)
# for ax in fg.axes.flatten():
#     ax.axhline(0,ls=':',c='red', alpha=0.7)
addTitleInfo(fg.axes.flat[0])
    
for i, ax in enumerate(fg.axes.flat):
    col_ = fg.col_names[i]
    if col_ == 'rnd':
        continue
    sp = np.array(palette)[hues[i]]
    sp = list(sp)
    sns.lineplot(data=df_[df_['ps_'] == col_], 
        x='trialwpertstage_wb', y='pred', 
        hue='pert_stage_wb', ax=ax, legend=None,
                palette = sp, dashes=[4,2])
    
print(fg.hue_kws, fg.hue_names)
fg.refline(y=0, color='red')
#fg.map(plt.hist, 'tip').refline(0.15)
    
fg.set_titles('{col_name}')
fg.set_xlabels('Trial number')
fg.set_ylabels('Error sensitivy')
    
lablet = 'A'
ax = fg.axes.flat[0]
ax.annotate(lablet, xy=(0, 1), xytext=(-60, 40), 
  fontsize=19, fontweight='bold', va='top', ha='left',
  xycoords='axes fraction', textcoords='offset points')
fnfig = pjoin(path_fig, f'Fig3_{lablet}_dynESpert_stage.pdf')
plt.savefig(fnfig)
plt.tight_layout()
plt.show()

###############################

palette=['blue', 'orange', 'green', 'olive','cyan','brown']
df_ = dfc.query('trial_shift_size == 1')
fg = sns.relplot(data=df_, kind='line',
            x='trialwpertstage_wb', col='ps_',
           y='err_sens_abserrcorr', hue='pert_stage_wb',
                 ci = 'sd', palette = palette,
           facet_kws={'sharex':False},
                 hue_order=hue_order,
            col_order = col_order, legend=None)
# for ax in fg.axes.flatten():
#     ax.axhline(0,ls=':',c='red', alpha=0.7)
addTitleInfo(fg.axes.flat[0])
    
for i, ax in enumerate(fg.axes.flat):
    col_ = fg.col_names[i]
    if col_ == 'rnd':
        continue
    sp = np.array(palette)[hues[i]]
    sp = list(sp)
    sns.lineplot(data=df_[df_['ps_'] == col_], 
        x='trialwpertstage_wb', y='ppred', 
        hue='pert_stage_wb', ax=ax, legend=None,
                palette = sp, dashes=[4,2])
    
print(fg.hue_kws, fg.hue_names)
fg.refline(y=0, color='red')

fg.set_titles('{col_name}')
fg.set_xlabels('Trial number')
fg.set_ylabels('Error sensitivy conditioned\non absolut error')


lablet = 'B'
ax = fg.axes.flat[0]
ax.annotate(lablet, xy=(0, 1), xytext=(-60, 40), 
  fontsize=19, fontweight='bold', va='top', ha='left',
  xycoords='axes fraction', textcoords='offset points')
fnfig = pjoin(path_fig, f'Fig3_{lablet}_dynESpert_stage.pdf')
plt.savefig(fnfig)
plt.tight_layout()
plt.show()

In [ ]:
gc.collect()

In [ ]:
#corrs_per_subj_me_errabs.info()

In [ ]:
df_[['ps_','pert_stage_wb','err_sens','trialwpertstage_wb',
    'subject']].info()

In [ ]:
dfc['ps_'].unique()

In [ ]:
#dfc[['err_sens', 'err_sens_abserrcorr']]

In [ ]:
df_ = dfc.query('trial_shift_size == 1 and environment == 0 and pert_stage_wb in [1,3]')
ax = sns.lineplot(x="trialwpertstage_wb", 
        y='err_sens',  estimator='mean',
                  #hue = 'pert_stage', # errorbar='se',
                  hue = 'pert_stage', 
         data=df_, color='#DE8F03')#, hue='trial_group_col_calc')
ax.legend(loc='lower right')
ax.axhline(0,ls=':',c='red', alpha=0.7)
ax.set_title('ES uncorr')
addTitleInfo(ax)
plt.show()

ax = sns.lineplot(x="trialwpertstage_wb", 
        y='err_sens_abserrcorr',  estimator='mean',
                  #hue = 'pert_stage', # errorbar='se',
                  hue = 'pert_stage', 
         data=df_, color='#DE8F03')#, hue='trial_group_col_calc')
ax.legend(loc='lower right')
ax.axhline(0,ls=':',c='red', alpha=0.7)
ax.set_title('ES corr by abs err')
addTitleInfo(ax)
plt.show()

ax = sns.lineplot(x="trialwpertstage_wb", 
        y='err_sens_errcorr',  estimator='mean',
                  #hue = 'pert_stage', # errorbar='se',
                  hue = 'pert_stage', 
         data=df_, color='#DE8F03')#, hue='trial_group_col_calc')
ax.legend(loc='lower right')
ax.axhline(0,ls=':',c='red', alpha=0.7)
ax.set_title('ES corr by err')
addTitleInfo(ax)

### on one axis

In [ ]:
p = [0,2,4]

df_ = dfc.query('trial_shift_size == 1 and environment == 0 and pert_stage_wb in @p')
ax = sns.lineplot(x="trialwpertstage_wb", 
        y='err_sens',  estimator='mean',
                  #hue = 'pert_stage', # errorbar='se',
                  hue = 'pert_stage', 
         data=df_, color='#DE8F03')#, hue='trial_group_col_calc')
ax.legend(loc='lower right')
ax.axhline(0,ls=':',c='red', alpha=0.7)
ax.set_title('ES uncorr')
addTitleInfo(ax)
plt.show()

ax = sns.lineplot(x="trialwpertstage_wb", 
        y='err_sens_abserrcorr',  estimator='mean',
                  #hue = 'pert_stage', # errorbar='se',
                  hue = 'pert_stage', 
         data=df_, color='#DE8F03')#, hue='trial_group_col_calc')
ax.legend(loc='lower right')
ax.axhline(0,ls=':',c='red', alpha=0.7)
ax.set_title('ES corr by abs err')
addTitleInfo(ax)
plt.show()

ax = sns.lineplot(x="trialwpertstage_wb", 
        y='err_sens_errcorr',  estimator='mean',
                  #hue = 'pert_stage', # errorbar='se',
                  hue = 'pert_stage', 
         data=df_, color='#DE8F03')#, hue='trial_group_col_calc')
ax.legend(loc='lower right')
ax.axhline(0,ls=':',c='red', alpha=0.7)
ax.set_title('ES corr by err')
addTitleInfo(ax)

In [ ]:
df_ = dfc.query('trial_shift_size == 1 and environment == 1')
ax = sns.lineplot(x="trialwb", 
        y='err_sens',  estimator='mean',
                  #hue = 'pert_stage', # errorbar='se',
                  hue = 'block_name', 
         data=df_, color='#DE8F03')#, hue='trial_group_col_calc')
ax.legend(loc='lower right')
ax.axhline(0,ls=':',c='red', alpha=0.7)
plt.show()

df_ = dfc.query('trial_shift_size == 1 and environment == 1')
ax = sns.lineplot(x="trialwb", 
        y='err_sens_abserrcorr',  estimator='mean',
                  #hue = 'pert_stage', # errorbar='se',
                  hue = 'block_name', 
         data=df_, color='#DE8F03')#, hue='trial_group_col_calc')
ax.legend(loc='lower right')
ax.axhline(0,ls=':',c='red', alpha=0.7)

In [ ]:
gc.collect()

# variability

In [ ]:
#dfc = dfall_aug.query('thr == "mestd*1.0"')
dfc = dfall_aug.query('thr == "mestd*1.0"')
dfc['trialwpertstage_wb'] = dfc['trialwpertstage_wb'].\
    where(dfc['environment'] == 0, dfc['trialwb'])
dfc['trialwpertstage_wb'] = dfc['trialwpertstage_wb'].astype(int)

dfc.duplicated(['subject','trials']).any()
# nan-ify after pause
dfc.loc[dfc['trialwb'] == 0, 'err_sens'] = np.nan

In [ ]:
def f(x):    
    if x > np.pi:
        x -= 2*np.pi
    elif x < -np.pi:
        x += 2*np.pi
    return x
dfc['error_pscadj'] = dfc['error_pscadj'].apply(f)

In [ ]:
dfc['error_change'] = dfc['error'] - dfc['error'].shift(1)

In [ ]:
dfcs = dfc.sort_values(['pert_seq_code', 'subject', 'trial_group_col_calc','trials'])
grp = dfcs.\
    groupby(['pert_seq_code', 'subject', 'trial_group_col_calc'] )
# grp = dfcs.\
#     groupby(['trial_group_col_calc','subject'] )



mavsz = 10
std_mavsz = 5
#vars_to_plot = ['err_sens','error', 'org_feedback']
vars_to_plot = ['err_sens','error_pscadj', 'error_change'] #, 'org_feedback_pscadj']
for varn in vars_to_plot:
    for g,gi in grp.groups.items():
        dfcs.loc[gi,f'{varn}_mav'] = dfcs.loc[gi,varn].rolling(mavsz).mean()    
        dfcs.loc[gi,f'{varn}_std'] = dfcs.loc[gi,varn].rolling(std_mavsz).std()    

In [ ]:
for varn in vars_to_plot:
    dfcs[f'{varn}_invstd'] = 1/dfcs[f'{varn}_std']

## corr with prev errors var

In [ ]:
#envv

## plot of var

In [ ]:
# plot VAR
use_var_pubnames = False
#%debug
for suff in ['std','invstd']:
    for varn, pertc in [('error_pscadj', 0.01), ('error_change',0.01), ('err_sens', 0.017)]:
        c = 1 / np.pi  * 180
        #for truncs, df in zip(['truncate q=5%','truncate no'],[dfc, dfc0]):
        for truncs, df in zip(['mestd * 1.0'],[dfcs]):
            plt.figure(figsize=(12,4))
            #qs = 'trial_group_col_calc in ["trialwe","trialwtgt_we"]'
            #qs += ' and pert_seq_code == 0'
            #df_= df.query(qs)
            df_ = df
            varn_eff = f"{varn}_{suff}"    
            ax = sns.lineplot(x="trials", y=varn_eff,  estimator='mean', ci='sd',
                         data=df_)
            vshift = 0.
            if varn.startswith('err_sens'):
                vshift = 0.5  
            # important to put this legend call before plotting pert and env
            #plt.legend(title='', loc='lower right', labels=['within tgt', 'across tgt'])
            ax.plot(tr, vshift + pert * pertc, c='violet', ls= '--', label='perturbation')
            #ax.plot(tr, vshift + envv * 30 * pertc, c='black', ls = ':', lw=0, marker='.' ,
            #        alpha=0.2, label='environment')

            axlbl = varn
            if use_var_pubnames:
                axlbl = var_pubnames[varn]
            else:
                ax.set_title(f'{varn}  {truncs}')

            ax.axhline(0,ls=':',c='red', alpha=0.7)
            ax.set_ylabel(axlbl + f'_{suff}' )
            addTitleInfo(ax)


In [ ]:
fg = sns.relplot(kind='line', x="trials", 
                 y='err_sens_mav',  estimator='mean', ci='sd',
                 col='subject', col_wrap=4,
                data=df_)
for ax in fg.axes.flatten():
    ax.plot(tr, vshift + pert * pertc, c='violet', ls= '--', label='perturbation')
    ax.axhline(0,ls=':',c='red', alpha=0.7)
        
plt.suptitle(f'err_sens_mav {mavsz}')
addTitleInfo(None)

In [ ]:
# plot MAV
use_var_pubnames = False
#%debug
for varn, pertc in [('error_pscadj', 0.01), 
                    ('error_change',0.001),
                    ('err_sens', 0.017)]:
    c = 1 / np.pi  * 180
    #for truncs, df in zip(['truncate q=5%','truncate no'],[dfc, dfc0]):
    for truncs, df in zip(['mestd * 1.0'],[dfcs]):
        plt.figure(figsize=(12,4))
        df_ = df
        varn_eff = f"{varn}_mav"  
        ax = sns.lineplot(x="trials", y=varn_eff,  estimator='mean', ci='sd',
                     data=df_)

        vshift = 0.
        if varn.startswith('err_sens'):
            vshift = 0.5
  
        # important to put this legend call before plotting pert and env
        #plt.legend(title='', loc='lower right', labels=['within tgt', 'across tgt'])
        ax.plot(tr, vshift + pert * pertc, c='violet', ls= '--', label='perturbation')
        #ax.plot(tr, vshift + envv * 30 * pertc, c='black', ls = ':', lw=0, marker='.' ,
        #        alpha=0.2, label='environment')
        
        axlbl = varn
        if use_var_pubnames:
            axlbl = var_pubnames[varn]
        else:
            ax.set_title(f'{varn}  {truncs}')
            
        ax.axhline(0,ls=':',c='red', alpha=0.7)
        ax.set_ylabel(axlbl + '_mav' )
        addTitleInfo(ax)
        

In [ ]:

#Modifying @jfb method, gives the function below which worked fine on ipykernel-5.3.4.
from IPython.display import display, Javascript
def getNotebookName():
    display(Javascript('IPython.notebook.kernel.execute("NotebookName = " + "\'"+window.document.getElementById("notebook_name").innerHTML+"\'");'))
    try:
        _ = type(NotebookName)
        return NotebookName
    except:
        return None
getNotebookName()

In [ ]:
dfall.columns

In [ ]:
dfall.groupby('subject')['movement_duration'].mean().mean()

In [ ]:
dfall.groupby('subject')['trial_duration'].mean().mean()

In [ ]:
plt.get_su

In [ ]:

os.environ.get("NOTEBOOK_PATH")

In [ ]:
from 

# Generalization (Fig 4)

In [ ]:
# Need to re-read entire dataset and don't limit to just one columns calc

In [ ]:
dfall_mshsz['thr']

In [ ]:
dfc_multi_tsz = dfall_mshsz.query('thr == "mestd*1.0"')
#dfc_multi_tsz = dfall_mshsz.query('thr == "mestd*0"')

In [ ]:
drs = dfc_multi_tsz['dist_rad_from_prevtgt'].unique();
drs_f = np.array( list(map( float, drs) ) )
drs = list(sorted(drs,key = lambda x: float(x)))

#dfc_s = dfc[dfc['trial_group_col_calc'] == 'trialwtgt_we']
# TODO: should I restrict to one env?
dfc_s = dfc_multi_tsz.query('trial_group_col_calc == "trialwe"')
grp = dfc_s.groupby(['trial_shift_size', 'dist_rad_from_prevtgt2'])
dts = dfc_multi_tsz['trial_shift_size'].unique()
nt = len(dts)

In [ ]:
drpairs = [(drs[i],drs[i+1] ) for i in range(3)]
drpairs += [(drs[0],drs[i] ) for i in range(2,4)]
drpairs += [(drs[1],drs[3] )]
print(drpairs)

dtpairs = [(dts[i],dts[i+1] ) for i in range(3)]
dtpairs += [(dts[0],dts[i] ) for i in range(2,4)]
dtpairs += [(dts[1],dts[3] )]
dtpairs

In [ ]:
def getPvals(dftmp,fitcol,pairs):
    from scipy.stats import ttest_ind
    pvalues = []
    for drp in pairs:
        if isinstance(drp[0],str):
            vs1 = dftmp.query(f'{fitcol} == "{drp[0]}"')['err_sens']
            vs2 = dftmp.query(f'{fitcol} == "{drp[1]}"')['err_sens']
        else:
            vs1 = dftmp.query(f'{fitcol} == {drp[0]}')['err_sens']
            vs2 = dftmp.query(f'{fitcol} == {drp[1]}')['err_sens']
        #ttr = ttest_ind(vs1,vs2)
        ttr = ttest_ind(vs1,vs2, alternative='greater')
        pvalues += [ttr.pvalue]
        
        print(drp, ttr.pvalue)
    # pvalues = [
    #     sci_stats.mannwhitneyu(robots, flight, alternative="two-sided").pvalue,
    #     sci_stats.mannwhitneyu(flight, sound, alternative="two-sided").pvalue,
    #     sci_stats.mannwhitneyu(robots, sound, alternative="two-sided").pvalue
    # ]

    # pvalues
    # [0.00013485140468088997, 0.2557331102364572, 0.00022985464929005115]

    # Transform each p-value to "p=" in scientific notation
    formatted_pvalues = [f'p={pvalue:.2e}' for pvalue in pvalues]
    return pvalues, formatted_pvalues

# make polynomial fits
def plotPolys(ax, dftmp, fitcol, degs=range(2,6), mean=1):
    if mean:
        me = dftmp.groupby(fitcol).median(numeric_only=1).reset_index()
        dftmp = me
    dftmp[fitcol] = pd.to_numeric(dftmp[fitcol] )
    esv, dv = dftmp[['err_sens',fitcol]]._values.T
    print(np.min(dv),dv,dv-np.min(dv),esv)
    #pr = np.polyfit(esv,dv,2)
    from numpy.linalg import LinAlgError
    dvu = np.unique(dv)
    dvu = np.array( list(sorted(dvu)) )
    print(dvu)
    for deg in degs:
        try:
            pr = np.polyfit(dv-np.min(dv),esv-np.min(esv),deg)        
        except (SystemError,LinAlgError):
            print(f'Failed deg={deg}')
            print(dv,esv, np.std(dv))
            continue
            
        poly = np.poly1d(pr)
        #if len(degs) > 1:
        if mean:
            lbl = f'polynomial fit of means deg={deg}'
        else:
            lbl = f'polynomial fit deg={deg}'
        #else:
        #    lbl = None
        esv2 = poly(dvu-np.min(dvu)) + np.min(esv)
        print(dvu-np.min(dvu), esv)
        ax.plot(range(len(dvu)) , poly(dvu-np.min(dvu)) + np.min(esv), 
                label=lbl, c='grey', lw=0.85 )
    return pr
    #ax.legend(loc='lower right')

#plotPolys(ax,dftmp,fitcol,mean=0)
ax = plt.gca()
plotPolys(ax,dftmp,fitcol, degs=rng, mean=meanfit)

In [ ]:
# pooled
#%debug

hue = 'env'
env2color = {'stable':'lightblue', 'random':'orange'}
aspect=1.3
#rng = [2]
#rng = [2,3,4,5,6,7]
rng = [2]
ylim = [-2,3]
meanfit=1
bnqs = [  ('grid_line_violin_spatial','Spatial distance [deg]',drs,drpairs,'dist_rad_from_prevtgt2', f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]} and trial_shift_size == 1'),
        ( 'grid_line_violin_temporal','Temporal distance [number of trials]',dts,dtpairs,'trial_shift_size','trial_group_col_calc == "trialwe" and dist_rad_from_prevtgt2 == "0.00"')]

for env_type in ['stable','random']:
    color = env2color[env_type]
    for bn,xlab,order,pairs,fitcol,qs in bnqs:
        #qs0 = f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]}'
        #qs = qs + ' and trial_shift_size == 1'
        if env_type in ['stable', 'random']:
            qs += ' and env == @env_type'
            hue = None

        dftmp0 = dfc_multi_tsz.query(qs)
        #fitcol = 'dist_rad_from_prevtgt2'
        fg = sns.catplot(kind='violin', data=dftmp0, y='err_sens', 
                         hue=hue, x=fitcol, order = order,
                          color=color, aspect=aspect, cut=True)
        dftmp = dfc_multi_tsz.query(qs).copy()
        pvalues, formatted_pvalues = getPvals(dftmp,fitcol, pairs)


        for ax in fg.axes.flatten():
            ax.axhline(0,ls=':',c='r')
            ax.set_ylabel('Error sensitivity', fontdict={'fontsize':12})
            ax.set_xlabel(xlab, fontdict={'fontsize':12} )
            if bn == 'grid_line_violin_spatial':
                ax.set_xticklabels(['same target', 45, 90, 135])
            ax.set_ylim(ylim)
            ax.set_title(f'{env_type} environment', y = 0.967,
                        fontdict={'fontsize':15, 'fontweight':'bold'})


#         from statannotations.Annotator import Annotator    
#         annotator = Annotator(ax,pairs, data=dftmp0, x=fitcol, y='err_sens',
#                              plot='violinplot', order=order)
#         #annotator.set_custom_annotations(formatted_pvalues)
#         annotator.set_pvalues(pvalues)
#         annotator.annotate()

        ax.set_ylim(-2,7)

        plotPolys(ax,dftmp,fitcol, degs=rng, mean=meanfit)
        #bn = 'grid_line_violin_spatial'
        plt.savefig(pjoin(path_fig,'behav',f'pool_{env_type}_{bn}.svg'))
        plt.savefig(pjoin(path_fig,'behav',f'pool_{env_type}_{bn}.pdf'))


In [ ]:
dfc_multi_tsz.groupby(['subject','environment','dist_rad_from_prevtgt2']).size()

In [ ]:
# not pooled
#%debug

hue = 'env'
env2color = {'stable':'lightblue', 'random':'orange'}
aspect=1.3
#rng = [2]
#rng = [2,3,4,5,6,7]
rng = [2]
ylim = [-2,3]
meanfit=1
bnqs = [  ('grid_line_violin_spatial','Spatial distance [deg]',drs,drpairs,'dist_rad_from_prevtgt2', f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]} and trial_shift_size == 1'),
        ( 'grid_line_violin_temporal','Temporal distance [number of trials]',dts,dtpairs,'trial_shift_size','trial_group_col_calc == "trialwe" and dist_rad_from_prevtgt2 == "0.00"')]

for env_type in ['stable','random']:
    color = env2color[env_type]
    for bn,xlab,order,pairs,fitcol,qs in bnqs:
        #qs0 = f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]}'
        #qs = qs + ' and trial_shift_size == 1'
        if env_type in ['stable', 'random']:
            qs += ' and env == @env_type'
            hue = None
            
        dftmp0 = dfc_multi_tsz.query(qs).groupby(['subject','environment',fitcol])
        dftmp0 = dftmp0.mean(numeric_only = True).reset_index()

        #dftmp0 = dfc_multi_tsz.query(qs)
        #fitcol = 'dist_rad_from_prevtgt2'
        fg = sns.catplot(kind='violin', data=dftmp0, y='err_sens', 
                         hue=hue, x=fitcol, order = order,
                          color=color, aspect=aspect, cut=True)
        #dftmp = dfc_multi_tsz.query(qs).copy()
        dftmp = dftmp0.copy()
        pvalues, formatted_pvalues = getPvals(dftmp,fitcol, pairs)


        for ax in fg.axes.flatten():
            ax.axhline(0,ls=':',c='r')
            ax.set_ylabel('Error sensitivity', fontdict={'fontsize':12})
            ax.set_xlabel(xlab, fontdict={'fontsize':12} )
            if bn == 'grid_line_violin_spatial':
                ax.set_xticklabels(['same target', 45, 90, 135])
            ax.set_ylim(ylim)
            ax.set_title(f'{env_type} environment', y = 0.967,
                        fontdict={'fontsize':15, 'fontweight':'bold'})


#         from statannotations.Annotator import Annotator    
#         annotator = Annotator(ax,pairs, data=dftmp0, x=fitcol, y='err_sens',
#                              plot='violinplot', order=order)
#         #annotator.set_custom_annotations(formatted_pvalues)
#         annotator.set_pvalues(pvalues)
#         annotator.annotate()

        ax.set_ylim(-1,1.5)

        plotPolys(ax,dftmp,fitcol, degs=rng, mean=meanfit)
        #bn = 'grid_line_violin_spatial'
        plt.savefig(pjoin(path_fig,'behav',f'nopool_{env_type}_{bn}.svg'))
        plt.savefig(pjoin(path_fig,'behav',f'nopool_{env_type}_{bn}.pdf'))


# MDS

In [ ]:
p = [1,3]
#p = [1]
df_ = df.query('pert_stage_wb in @p')

df_['pert_stage'] = df_['pert_stage'].astype('int')
df_['trialwpertstage_wb'] = df_['trialwpertstage_wb'].astype('int')
df_['pert_stage'] = df_['pert_stage'].astype('category')



In [ ]:
df_['error_pscadj_pertstageadj_deg'] = df['error_pscadj_pertstageadj'] * 180/np.pi

### is last pert within subj worse than other perts?

In [ ]:
fg = sns.relplot(data=df_,kind='line', x='trialwpertstage_wb', 
             y='error_pscadj_pertstageadj_deg', hue='pert_stage', col='subject',
        col_wrap=4, palette=['r','g','b','k'])
for ax in fg.axes.flatten():
    ax.set_ylim(-30,50)
    ax.axhline(y=0,c='red',ls=':')

In [ ]:
# 03, 04, 07, 

except 1,6 and 3,6  further adaptatino is worse than first previous one
1 is comparable to 6 and  6 is better than 3

so very first pert stage is better than anyone except 6 (first pert stage after random)
also second pert stage is better than fourth (second after random)
also third (first pert after random) is better than second and fourth

so first perts (before and after random) are the best,  
second are worse, the last one is absolute worst

In [ ]:
col

In [ ]:
ttrs = []

for i1,i2 in [( 1,3), (1,6), (1,8), (6,3), (3,8), (6,8)]:
    dfdif = df_.query('pert_stage == @i1').set_index(['subject','trialwpertstage_wb'])[col] -\
        df_.query('pert_stage == @i2').set_index(['subject','trialwpertstage_wb'])[col]
    
    ttr = pg.ttest(dfdif,0, alternative='less' )
    ttr['i1'] = i1; ttr['i2'] = i2
    ttr['type'] = 'all_pooled'
    ttrs += [ttr]
    
    # mean across subjects within trial
    mesubj = dfdif.to_frame().reset_index().groupby('trialwpertstage_wb').mean()
    ttr = pg.ttest(mesubj[col],0, alternative='less' )
    ttr['i1'] = i1; ttr['i2'] = i2
    ttr['type'] = 'mean_over_subjects'
    ttrs += [ttr]
    
    # mean across trials within subj
    mewsubj = dfdif.to_frame().reset_index().groupby('subject').mean()
    ttr = pg.ttest(mewsubj[col],0, alternative='less' )
    ttr['i1'] = i1; ttr['i2'] = i2
    ttr['type'] = 'mean_over_trials'
    ttrs += [ttr]

    print(i1,i2)
    display(mesubj.describe())

ttrs = pd.concat(ttrs)

In [ ]:
print(getAddInfo())
display(ttrs[['T','p-val','i1','i2','type']])

In [ ]:
pert_stage

In [ ]:
#df_['error_pscadj_pertstageadj']

### mds plots

In [ ]:
pivoted_df = df_.pivot(index=["trialwpertstage_wb", 'subject'],
                      columns=["pert_stage"], values="error")

# convert the pivoted DataFrame to a numpy array
X = pivoted_df.to_numpy()
print(X.shape)

# perform MDS with 2 components
mds = MDS(n_components=2, random_state=0)
Y = mds.fit_transform(X)

In [ ]:
# one subj = 1 point
pivot_df = df_.pivot_table(index=["subject", "pert_stage", "trialwpertstage_wb"], 
                           values="error_pscadj_pertstageadj")

# convert the pivot table to a 2D numpy array
pivot_array0 = pivot_df.to_numpy()

subjects = df_["subject"].unique()
trialwpertstage_wb = df_["trialwpertstage_wb"].unique()
pert_stage = df_["pert_stage"].unique()
pivot_array = np.reshape(pivot_array0, 
    (len(subjects), len(trialwpertstage_wb), len(pert_stage)))

# print the shape and the content of the array
print(pivot_array.shape)
print(pivot_array)

#Xrsh = X.reshape(20,-1,4)
Xrsh = pivot_array


X = pivot_array.reshape(-1,4*60)
print(X.shape)
mds = MDS(n_components=2, random_state=0)
Y_complete_subj = mds.fit_transform(X)
print(Y_complete_subj.shape)

In [ ]:
Xrsh.shape

In [ ]:
Yrsh.shape

In [ ]:
#Y
Yrsh = Y_complete_subj
# create a figure with one subplot
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

# plot the 2D projection of the trajectory with different colors for each subject
#colors = ["red", "blue", "green", "yellow"] # define a list of colors for each subject
#for subject in range(1, 5):
S = 40
N = 10
for si in np.arange(N):
    # select the rows that belong to the current subject
    #mask = df["subject"] == subject
    ax.scatter(Yrsh[si,0], Yrsh[si,1], s=S)
               #, color=colors[subject-1], label=f"Subject {subject}")
#ax.set_xlim(-1,1)
#ax.set_ylim(-1,1)

# add title and labels to the plot
ax.set_title("MDS Projection of Error Trajectory")
ax.set_xlabel("MDS Component 1")
ax.set_ylabel("MDS Component 2")

# add a legend to show the color mapping
#ax.legend()

# show the figure
plt.show()

In [ ]:
a4d[:,:,:,0].reshape(-1, 60).T

In [ ]:
# # one pert stage = 1 point
# pivot_df = df_.pivot_table(index=["subject", "pert_stage", "trialwpertstage_wb"], 
#                            values="error_pscadj_pertstageadj")

# # convert the pivot table to a 2D numpy array
# pivot_array0 = pivot_df.to_numpy()

# subjects = df_["subject"].unique()
# trialwpertstage_wb = df_["trialwpertstage_wb"].unique()
# pert_stage = df_["pert_stage"].unique()
# pivot_array = np.reshape(pivot_array0, 
#     (len(subjects), len(trialwpertstage_wb), len(pert_stage)))

# # print the shape and the content of the array
# print(pivot_array.shape)
# #print(pivot_array)

# #Xrsh = X.reshape(20,-1,4)
# Xrsh = pivot_array


# X = pivot_array.reshape(-1,60, order='F')

Xrsh = a4d[:,:,:,1]
print(Xrsh.shape)
#np.rollaxis(Xrsh,1).shape
X = Xrsh.reshape(-1, 60)
print(X.shape)

mds = MDS(n_components=2, random_state=0)
Y_complete = mds.fit_transform(X)
print(Y_complete.shape)

Yrsh.shape

#Y
Yrsh = Y_complete.reshape(20,4,2)
# create a figure with one subplot
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# plot the 2D projection of the trajectory with different colors for each subject
#colors = ["red", "blue", "green", "yellow"] # define a list of colors for each subject
#for subject in range(1, 5):
S = 80
N = 20
for si in np.arange(N):
    # select the rows that belong to the current subject
    #mask = df["subject"] == subject
    #for mi,m in enumerate(['+','x','o','*']):
    #    ax.scatter(Yrsh[si,mi, 0], Yrsh[si,mi, 1], s=S, alpha=0.7, marker=m)
    plt.plot(Yrsh[si,:, 0], Yrsh[si,:, 1])
               #, color=colors[subject-1], label=f"Subject {subject}")
#ax.set_xlim(-1,1)
#ax.set_ylim(-1,1)

# add title and labels to the plot
ax.set_title("MDS Projection of Error Trajectory")
ax.set_xlabel("MDS Component 1")
ax.set_ylabel("MDS Component 2")

# add a legend to show the color mapping
#ax.legend()

# show the figure
plt.show()

In [ ]:
pivot_df = df_.pivot_table(index=["subject", "trialwpertstage_wb", "pert_stage"], 
                           values="error_pscadj_pertstageadj")

# convert the pivot table to a 2D numpy array
pivot_array0 = pivot_df.to_numpy()

subjects = df_["subject"].unique()
trialwpertstage_wb = df_["trialwpertstage_wb"].unique()
pert_stage = df_["pert_stage"].unique()
pivot_array = np.reshape(pivot_array0, 
    (len(subjects), len(trialwpertstage_wb), len(pert_stage)))

# print the shape and the content of the array
print(pivot_array.shape)
#print(pivot_array)

#Xrsh = X.reshape(20,-1,4)
Xrsh = pivot_array

In [ ]:
a4d = a3d.reshape(20,4,60,2, order='F')
X = np.rollaxis(a4d[:,:,:,1],2)
a = np.rollaxis(a4d[:,:,:,0],2)
print(X.shape)# [:,:,0]
#X = a4d[:,:,:,1].reshape()

In [ ]:
a

In [ ]:
mds = MDS(n_components=2, random_state=0)
Y = mds.fit_transform(X.reshape(-1,4,order='F'))#.reshape( *X.shape[:2], -1)

In [ ]:
Yrsh = Y.reshape( *X.shape[:2], -1, order='F')
print(Y.shape, Yrsh.shape)

In [ ]:
# Yrsh = np.rollaxis(Yrsh,1)
# Yrsh.shape

In [ ]:
X = pivot_array.reshape(-1,4, order='F')
print(X.shape)
mds = MDS(n_components=2, random_state=0)
Y = mds.fit_transform(X)
print(Y.shape)

In [ ]:
# TODO: other data groupings (single subject as datapoint, entire pert stage as datapoint)

In [ ]:
#Y
print(Yrsh.shape)
#Yrsh = Y.reshape(20,-1,2, order = 'F')
# create a figure with one subplot
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# plot the 2D projection of the trajectory with different colors for each subject
#colors = ["red", "blue", "green", "yellow"] # define a list of colors for each subject
#for subject in range(1, 5):
S = 40
N = 2
for si in np.arange(N):
    # select the rows that belong to the current subject
    #mask = df["subject"] == subject
    ax.scatter(Yrsh[:,si, 0], Yrsh[:,si, 1], s=S, alpha=0.7)
               #, color=colors[subject-1], label=f"Subject {subject}")
#ax.set_xlim(-1,1)
#ax.set_ylim(-1,1)

# add title and labels to the plot
ax.set_title("MDS Projection of Error Trajectory")
ax.set_xlabel("MDS Component 1")
ax.set_ylabel("MDS Component 2")

# add a legend to show the color mapping
#ax.legend()

# show the figure
plt.show()

In [ ]:
a3d = np.array(list(df_.groupby(['subject','pert_stage']).apply(lambda x:
    x[['trialwpertstage_wb','error_pscadj_pertstageadj']].to_numpy())))
print(a3d.shape)
#subj, pertstage, trial, trialind
a4d = a3d.reshape(20,4,60,2, order='F')
a4d_ = a3d.reshape(20,60,4,2, order='F')

In [ ]:
a4d[0,:,:,0]

In [ ]:
plt.plot(a4d[0,:,:,1].T)
plt.show()
plt.plot(a4d[1,:,:,1].T)

In [ ]:
#X
Xrsh = a4d[:,:,:,1]
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
N = 3
S = 10
pargs={'marker':'o', 'ms':S, 'lw':0, 'alpha':0.5}
ax = axs[0,0]
i1,i2 = 0,1
for si in np.arange(N):
    #ax.plot(Xrsh[si,:, 0], Xrsh[si,:, 1], ms=S)
    ax.plot(Xrsh[si,i1,:], Xrsh[si,i2,:], **pargs)
    ax.set_xlabel(str(i1)); ax.set_ylabel(str(i2)); 

ax = axs[0,1]
i1,i2 = 2,3
for si in np.arange(N):
    ax.plot(Xrsh[si,i1,:], Xrsh[si,i2,:], **pargs)
    ax.set_xlabel(str(i1)); ax.set_ylabel(str(i2)); 
    
ax = axs[1,0]
i1,i2 = 0,2
for si in np.arange(N):
    ax.plot(Xrsh[si,i1,:], Xrsh[si,i2,:], **pargs)
    ax.set_xlabel(str(i1)); ax.set_ylabel(str(i2)); 

ax = axs[1,1]
i1,i2 = 1,3
for si in np.arange(N):
    ax.plot(Xrsh[si,i1,:], Xrsh[si,i2,:], **pargs)
    ax.set_xlabel(str(i1)); ax.set_ylabel(str(i2)); 

######################
ylim = (-1,1)
for ax in axs.flatten():
    ax.set_xlim(*ylim)
    ax.set_ylim(*ylim)

# show the figure
plt.tight_layout()
plt.show()

In [ ]:
sns.scatterplot(x='')

In [ ]:
#This means that the shape of the array will be (nrows, ncols * nvalues),
pivot_df = df_.pivot_table(index=["subject", 'trialwpertstage_wb'], 
                           columns=[ "pert_stage"],
                           values=["error_pscadj_pertstageadj"])
pivot_df.to_numpy().shape

In [ ]:
pivot_df

In [ ]:
pivot_df.values.shape

In [ ]:
df__ = df_[["subject", "pert_stage", "trialwpertstage_wb",
     'error_pscadj_pertstageadj']].set_index(["subject", "pert_stage", "trialwpertstage_wb"])
df__.unstack(level=-1).to_numpy().shape #['error_pscadj_pertstageadj']
#df_.set_index(["subject", "pert_stage", "trialwpertstage_wb"])['error_pscadj_pertstageadj'].unstack(-1).to_numpy().shape
#values.shape

In [ ]:
df__['error_pscadj_pertstageadj'].unstack().to_numpy().shape

In [ ]:
80*60

In [ ]:
pivot_df = df_.pivot_table(index=["subject","trialwpertstage_wb"], 
                           columns=[ "pert_stage"],
                           values=["error_pscadj_pertstageadj"], 
                           aggfunc='first')
pivot_df

In [ ]:
# one pert stage = 1 point
pivot_df = df_.pivot_table(index=["subject", "pert_stage"], 
                           columns='trialwpertstage_wb',
                           values=["error_pscadj_pertstageadj"], aggfunc='first')

# convert the pivot table to a 2D numpy array
pivot_array0 = pivot_df.to_numpy()

subjects = df_["subject"].unique()
trialwpertstage_wb = df_["trialwpertstage_wb"].unique()
pert_stage = df_["pert_stage"].unique()
pivot_array = np.reshape(pivot_array0, 
    (len(subjects), len(trialwpertstage_wb), len(pert_stage)), order='F')
pivot_array.shape

In [ ]:
pivot_df.values.shape

In [ ]:
pivot_df

In [ ]:
print(pivot_array0.shape, pivot_array.shape)
pivot_array[0,:,0]

In [ ]:
pd.__version__


In [ ]:
df00 = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie', 'David'],
    'type': ['A', 'B', 'A', 'B'],
    'date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04'],
    'value': [10, 20, 30, 40]
})

pivoted = df00.pivot(index='name', columns=['type', 'date'], values='value')
pivoted.values.shape